In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [3]:
!pip install tensorflow==2.2.0 --user
!pip install gym==0.18.3 --user
!pip install stable-baselines3[extra]==1.0 --user

In [1]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name)

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:44.0
Episode:2 Score:13.0
Episode:3 Score:9.0
Episode:4 Score:16.0
Episode:5 Score:11.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [10]:
# 0-push cart to left, 1-push cart to the right
env.action_space

Discrete(2)

In [9]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space

Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)

# 3. Train an RL Model

In [7]:
import os
log_path = os.path.join('Training','Logs')

env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [8]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 2297 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1292        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010800737 |
|    clip_fraction        | 0.1         |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00318    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.27        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0147     |
|    value_loss           | 49.2        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [12]:
import os
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [13]:
model.save(PPO_path)

In [14]:
del model

In [15]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [16]:
from stable_baselines3.common.evaluation import evaluate_policy

In [17]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\marti\anaconda3\envs\RL_3h\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(200.0, 0.0)

In [18]:
env.close()

# 5. Test Model

In [22]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break
        
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.18207535, -0.73765454,  0.03268928,  0.51031907])}]
Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


In [23]:
env.close()

# 6. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [24]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [25]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [26]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [27]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [28]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [29]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 425  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 590         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009418713 |
|    clip_fraction        | 0.0917      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.0054     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.72        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0136     |
|    value_loss           | 51.6        |
-----------------------------------------
---

C:\Users\marti\anaconda3\envs\RL_3h\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=10000, episode_reward=186.60 +/- 17.27
Episode length: 186.60 +/- 17.27
New best mean reward!
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 187        |
|    mean_reward          | 187        |
| time/                   |            |
|    fps                  | 756        |
|    iterations           | 5          |
|    time_elapsed         | 13         |
|    total_timesteps      | 10240      |
| train/                  |            |
|    approx_kl            | 0.01197375 |
|    clip_fraction        | 0.0781     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.606     |
|    explained_variance   | 0.228      |
|    learning_rate        | 0.0003     |
|    loss                 | 35.4       |
|    n_updates            | 40         |
|    policy_gradient_loss | -0.0165    |
|    value_loss           | 70.2       |
----------------------------------------
--------------------------

In [30]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [41]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.3, 0.90000004)

In [42]:
env.close()

# 8. Changing Policies

In [33]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [34]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [35]:
model.learn(total_timesteps=20000, callback=eval_callback)

------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    fps                  | 1166         |
|    iterations           | 1            |
|    time_elapsed         | 1            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0041018757 |
|    clip_fraction        | 0.0367       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.548       |
|    explained_variance   | 0.508        |
|    learning_rate        | 0.0003       |
|    loss                 | 26.1         |
|    n_updates            | 90           |
|    policy_gradient_loss | -0.00416     |
|    value_loss           | 85.5         |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps     

# 9. Using an Alternate Algorithm

In [38]:
from stable_baselines3 import DQN

In [39]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [40]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration rate | 0.952    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7215     |
|    time_elapsed     | 0        |
|    total timesteps  | 101      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.902    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 8584     |
|    time_elapsed     | 0        |
|    total timesteps  | 206      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.864    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 8528     |
|    time_elapsed     | 0        |
|    total timesteps  | 286      |
----------------------------------
------------------------

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()